# CloudML
- [구글 Cloud ML](https://cloud.google.com/ml-engine/)
- 확장 가능한 관리형 기계학습 : 데이터의 유형과 크기와 관계없이 기계학습 모델을 손쉽게 제작할 수 있는 서비스
- TensorFlow 프레임워크를 사용
- 학습된 모델을 즉시 사용 가능
- 사전 처리를 위해 [Google Data Flow](https://cloud.google.com/dataflow/)에 통합되어 있음
- 온라인 / 일괄 예측 서비스가 존재
- Hypter Tune 기능을 사용해 자동으로 모델 학습을 조정해 나은 결과를 얻을 수 있음
- Datalab을 사용해 모델을 쉽게 제작할 수 있음
- GPU 가속이 포함된 확장 가능한 분산 학습 인프라


### 정리
- Google Dataflow, Cloud Storage, Datalab과 함께 작동
- HyperTune : 모델에 맞는 값을 직접 찾지 않고 HypterTune 기능으로 자동으로 튜닝
- 관리형 서비스 : 인프라에 대한 걱정 없이 모델 개발과 예측에 집중
- 확장 가능한 서비스 : CPU / GPU를 지원하는 관리형 분산 학습 인프라
- 노트형 개발자 환경 : jupyter 노트북과 같은 Datalab을 사용
- 이식 가능한 모델 : TensorFlow SDK를 이용해 샘플 데이터세트에서 로컬로 모델을 학습시키고 규모에 맞게 학습에 사용 가능. Cloud Machine Learning Engine으로 학습시킨 모델은 로컬 실행이나 모바일 통합을 위해 다운로드할 수 있음
- [가격 계산 문서](https://cloud.google.com/products/calculator/)를 통해 가격 예측 가능
- python2만 지원

### 가격
- [ML Engine 가격 문서](https://cloud.google.com/ml-engine/pricing)
<img src="./img/google_cloud_mlengine_01.png" width="700" height="700">


# 1-1. Quick start using Command-Line
### Cloud Shell 이용시
- 구글 클라우드 플랫폼의 클라우드 쉘 클릭
- 쉘에서 아래와 같은 명령어 입력
~~~
gcloud config set project project_id
~~~
- 설치가 되었는지 확인하기
~~~
gcloud ml-engine models list
~~~
- Listed 0 items가 나오면 성공

### mac 이용시
- virtualenv로 가상환경 설정
~~~
pip install virtualenv
virtualenv -p python2 env_name
source env_name/bin/activate
~~~
- [gcloud sdk 설치](https://cloud.google.com/sdk/docs/quickstart-mac-os-x#before-you-begin)
~~~
gcloud init
pip install -U pip
pip install --upgrade tensorflow
gcloud auth application-default login
~~~
- 설치가 되었는지 확인하기
~~~
gcloud ml-engine models list
~~~
- Listed 0 items가 나오면 성공

# 1-2. Quick start using datalab
- [Datalab Quick Start](https://cloud.google.com/datalab/docs/quickstarts) 를 진행하고 와주세요
- datalab connect instance명으로 datalab에 접속합니다
- [BigQuery with datalab](https://github.com/zzsza/bigquery-tutorial/blob/master/tutorials/02-Utils/02.%20Connect%20Datalab.ipynb) 문서도 참고하시면 좋습니다

# 2. Get Start
- 터미널에서 아래와 같이 입력해서 데이터를 받습니다
~~~
wget https://github.com/GoogleCloudPlatform/cloudml-samples/archive/master.zip
unzip master.zip
cd cloudml-samples-master/census/estimator
~~~

- Cloud Machine Learning Engine : Training / Prediction
- Cloud Storage : Input data 저장, trainer package 스테이징, training 구조 작성, 배치 예측을 위한 input 데이터 파일 저장



## 2-1 Local에서 검증 작업
- 클라우드에 올리기 전에, Local에서 테스트해봅니다. (테스트를 꼭 해봐야 필요없는 비용이 부과 안되요..! )

### 가상환경 구축하기
- cloudml-samples-master/census 위치에서 진행해주세요! ( 터미널 입력 )
- virtualenv 가 깔려있지 않다면 pip install virtualenv로 설치!
~~~
virtualenv --python=python env
~~~

### Data 받아오기
- 터미널에서 census/estimator 폴더에 진입한 후, 아래와 같은 명령어를 입력해주세요 ( 터미널!! )

~~~
mkdir data
gsutil cp gs://cloudml-public/census/data/* ./data/

혹시 no matches found: gs://cloudml-public/census/data/* 오류가 나오면 아래 코드를 입력해주세요
gsutil cp 'gs://cloudml-public/census/data/*' ./data/
~~~

### TRAIN_DATA, EVAL_DATA 경로 설정
- 위와 마찬가지로 cloudml-samples-master/census/estimator에 진입하신 후 데이터의 경로를 설정해주세요
~~~
TRAIN_DATA=$(pwd)/data/adult.data.csv
EVAL_DATA=$(pwd)/data/adult.test.csv
~~~



#### 환경 설정 불러오기
- 위에서 virtualenv로 구축한 환경 설정을 불러와주세요
- (cloudml-samples-master/census 위치에서)

~~~
source env/bin/activate
~~~

#### requirements.txt 의 라이브러리 설치

~~~
sudo pip install -r ../requirements.txt
~~~

- six 관련 오류가 발생시 아래와 같이 입력해주세요

~~~
sudo pip install haxor-news --upgrade --ignore-installed six
~~~

- macos일 경우 맥 기본 내장 python이 아닌, homebrew로 python를 설치하신 후 환경설정해주셔야 정상적으로 설치가 됩니다

#### output 디렉토리 설정

~~~
mkdir output
MODEL_DIR=output
~~~

- 이전 학습 결과가 남아있을 수 있으므로 아래와 같은 명령어로 폴더 내부 파일을 삭제해줍니다

~~~
rm -rf $MODEL_DIR/*
~~~


## train 시작!
~~~
gcloud ml-engine local train \
    --module-name trainer.task \
    --package-path trainer/ \
    -- \
    --train-files $TRAIN_DATA \
    --eval-files $EVAL_DATA \
    --train-steps 1000 \
    --job-dir $MODEL_DIR \
    --eval-steps 100 \
    --verbosity DEBUG
~~~

- module-name : 학습시킬 모델 ( 위 코드에선 trainer 폴더 내의 task.py )
- package-path : python 모듈로 지정할 경로
- -- \ 라고 되어있는 것 밑부분은 task.py에서( module ) 사용할 파라미터들



### tensorboard 실행
~~~
python -m tensorflow.tensorboard --logdir=output
~~~

- 종료는 control + c

### local run in distributed mode ( 분산 모드 )

~~~
gcloud ml-engine local train \
    --module-name trainer.task \
    --package-path trainer/ \
    --distributed \
    -- \
    --train-files $TRAIN_DATA \
    --eval-files $EVAL_DATA \
    --train-steps 1000 \
    --job-dir $MODEL_DIR
~~~

- 결과 확인

~~~
ls -R output-dist/
~~~

- 텐서보드

~~~
python -m tensorflow.tensorboard --logdir=$MODEL_DIR
~~~

## 2-2 Cloud ML에 올리기
#### Cloud Bucket 생성
- [Bucket 생성](https://cloud.google.com/ml-engine/docs/working-with-data#using_a_cloud_storage_bucket_from_a_different_project)

###  기본 bucket 설정
~~~
PROJECT_ID=$(gcloud config list project --format "value(core.project)")
AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT_ID}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print response['serviceAccount']")
~~~


~~~
BUCKET_NAME="your_bucket_name"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET_NAME
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET_NAME
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET_NAME

REGION=asia-east1
~~~

### 위에서 설정한 bucket_name 확인
~~~
echo $BUCKET_NAME
~~~

### 로컬의 파일을 gs로 올리기
~~~
gsutil cp -r data gs://$BUCKET_NAME/data
TRAIN_DATA=gs://$BUCKET_NAME/data/adult.data.csv
EVAL_DATA=gs://$BUCKET_NAME/data/adult.test.csv
gsutil cp ../test.json gs://$BUCKET_NAME/data/test.json
TEST_JSON=gs://$BUCKET_NAME/data/test.json
~~~

### job 이름과 output 경로 설정

~~~
JOB_NAME=census_single_1
OUTPUT_PATH=gs://$BUCKET_NAME/$JOB_NAME
~~~

### run!

~~~
gcloud ml-engine jobs submit training $JOB_NAME \
--job-dir $OUTPUT_PATH \
--runtime-version 1.2 \
--module-name trainer.task \
--package-path trainer/ \
--region $REGION \
-- \
--train-files $TRAIN_DATA \
--eval-files $EVAL_DATA \
--train-steps 1000 \
--verbosity DEBUG
~~~

- --runtime-version 1.2라고 tensorflow 버전을 꼭 명시하는 습관을 가지는 것이 좋습니다

### 결과 보기
- [콘솔](https://console.cloud.google.com)에서 ML Engine -> Jobs -> View logs

## 분산으로 Cloud ML에서 진행하고 싶다면
- --scale-tier 옵션을 추가하면 됩니다! [공식 문서](https://cloud.google.com/ml-engine/reference/rest/v1/projects.jobs#scaletier)

~~~
gcloud ml-engine jobs submit training $JOB_NAME \
    --job-dir $OUTPUT_PATH \
    --runtime-version 1.2 \
    --module-name trainer.task \
    --package-path trainer/ \
    --region $REGION \
    --scale-tier STANDARD_1 \
    -- \
    --train-files $TRAIN_DATA \
    --eval-files $EVAL_DATA \
    --train-steps 1000 \
    --verbosity DEBUG  \
    --eval-steps 100
~~~

## 3. 하이퍼 파라미터 튜닝
- [공식 문서](https://cloud.google.com/ml-engine/docs/hyperparameter-tuning-overview)
- 하이퍼 파라미터 설정 파일 (yaml 파일)이 존재해야 합니다

- census의 예시 파일

~~~
trainingInput:
  hyperparameters:
    goal: MAXIMIZE
    hyperparameterMetricTag: accuracy
    maxTrials: 4
    maxParallelTrials: 2
    params:
      - parameterName: first-layer-size
        type: INTEGER
        minValue: 50
        maxValue: 500
        scaleType: UNIT_LINEAR_SCALE
      - parameterName: num-layers
        type: INTEGER
        minValue: 1
        maxValue: 15
        scaleType: UNIT_LINEAR_SCALE
      - parameterName: scale-factor
        type: DOUBLE
        minValue: 0.1
        maxValue: 1.0
        scaleType: UNIT_REVERSE_LOG_SCALE

~~~

### 기본 설정

~~~
HPTUNING_CONFIG=../hptuning_config.yaml
JOB_NAME=census_core_hptune_1
TRAIN_DATA=gs://$BUCKET_NAME/data/adult.data.csv
EVAL_DATA=gs://$BUCKET_NAME/data/adult.test.csv
OUTPUT_PATH=gs://$BUCKET_NAME/$JOB_NAME
~~~

### Run

~~~
gcloud ml-engine jobs submit training $JOB_NAME \
    --stream-logs \
    --job-dir $OUTPUT_PATH \
    --runtime-version 1.2 \
    --config $HPTUNING_CONFIG \
    --module-name trainer.task \
    --package-path trainer/ \
    --region $REGION \
    --scale-tier STANDARD_1 \
    -- \
    --train-files $TRAIN_DATA \
    --eval-files $EVAL_DATA \
    --train-steps 1000 \
    --verbosity DEBUG  \
    --eval-steps 100
~~~